In [1]:
import cv2
import numpy as np
import pandas as pd
from patchify import patchify
from skimage.io import imread, imshow, imsave
from keras.utils.all_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout, Flatten, Dense
from keras.models import Model, load_model
from keras.optimizers import adam_v2
from keras.losses import binary_crossentropy
from keras.applications.resnet import ResNet50
from keras.metrics import Precision, Recall
from sklearn.utils.class_weight import compute_class_weight

In [4]:
train_img_dir = 'C:/ML/RS/train_dataset_train/train'
size = 512
num_classes = 69

X = []
Y = []
for_weight = []
data_df = pd.read_csv("C:/ML/RS/train_dataset_train/train.csv")
labels = pd.DataFrame(columns=['id', 'img', 'sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8'])
for i in range(len(data_df)):
    line = data_df.iloc[i]
    for_weight.append([line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9]])
    img = imread(train_img_dir + '/' + line[1])
    img = cv2.resize(img, dsize = [size, size], interpolation=cv2.INTER_NEAREST)
    X.append(img)
    lb = [line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9]]
    cat_lb = to_categorical(lb, num_classes=num_classes+1, dtype='uint8')
    out_lb = cat_lb[0] + cat_lb[1] + cat_lb[2] + cat_lb[3] + cat_lb[4] + cat_lb[5] + cat_lb[6] + cat_lb[7]
    out_lb = np.delete(out_lb, 0)
    Y.append(out_lb)
X = np.array(X)
Y = np.array(Y)
for_weight = np.array(for_weight)

In [5]:
for_weight_flatten = for_weight.reshape(for_weight.shape[0]*for_weight.shape[1])
weight_array = compute_class_weight(class_weight='balanced', classes=np.unique(for_weight_flatten), y=for_weight_flatten)

weight_dict = dict(zip(np.unique(for_weight_flatten), weight_array))
del weight_dict[0]
weight_dict.update({53: 1.0})
weight_dict.update({64: 1.0})
weight_dict = dict(sorted(weight_dict.items()))
new_dict = {}
for i in range(len(weight_dict)):
    new_dict[i] = weight_dict[i+1]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [7]:
base_model = ResNet50(weights='imagenet', input_shape=(size,size,3), include_top=False)
base_out = base_model.output

In [8]:
x = base_out
x = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
x = Flatten()(x)
num_neurons = x.shape[1]
x = Dense(num_neurons, activation = 'relu')(x)
x = Dense(1024, activation = 'relu')(x)
outputs = Dense(num_classes, activation = 'sigmoid')(x)

In [9]:
model = Model(inputs=base_model.inputs, outputs=outputs)

In [ ]:
model.compile(optimizer=adam_v2.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=["accuracy", Recall()])
model.fit(X_train, Y_train, epochs=1, batch_size=4, class_weight=new_dict, validation_data=(X_test, Y_test), verbose=1)

 23/146 [===>..........................] - ETA: 11:14 - loss: 0.6668 - accuracy: 0.0543 - recall: 0.0426

In [42]:
test_img_dir = 'C:/ML/RS/test_dataset_test/test'
treshold = 0.75
out_path = 'C:/ML/RS/Results/attempt_2/'

data_df_test = pd.read_csv("C:/ML/RS/test_dataset_test/test.csv")
labels = pd.DataFrame(columns=['id', 'sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8'])
for i in range(len(data_df_test)):
    line = data_df_test.iloc[i]
    img = imread(test_img_dir + '/' + line[1])
    img = cv2.resize(img, dsize = [size, size], interpolation=cv2.INTER_NEAREST)
    test_img_input = np.expand_dims(img, 0)
    prediction = np.squeeze(model.predict(test_img_input))
    prediction[prediction>=treshold] = 1
    prediction[prediction<treshold] = 0
    prediction = prediction.astype('uint8')
    indx = [0, 0, 0, 0, 0, 0, 0, 0]
    count = 0
    for num, value in enumerate(prediction):
        if value == 1:
            indx[count] = num + 1
            count += 1
            if count > 7:
                break
    new_line = {'id'   :line[0],
                'sing1':indx[0],
                'sing2':indx[1],
                'sing3':indx[2],
                'sing4':indx[3],
                'sing5':indx[4],
                'sing6':indx[5],
                'sing7':indx[6],
                'sing8':indx[7]}
    labels = labels.append(new_line, ignore_index=True)
labels.to_csv(out_path + 'Out_labels_2_1.csv', index=False)